# STEP 3: Funzione di Inferenza Unica

Funzione che combina classificatore OCCLUSION e autoencoder per classificare
un'immagine come "OK", "KO" o "OCCLUSION".

**Pipeline:**
1. STEP 1 - Occlusione: Verifica se l'immagine è occlusa → se sì, ritorna "OCCLUSION"
2. STEP 2 - Anomaly Detection: Se visibile, verifica se è anomalo → se errore > threshold → "KO", altrimenti "OK"


In [ ]:
# Setup: Clona repository GitHub e monta Google Drive per i dati
import os
from pathlib import Path

# Opzione 1: Clona da GitHub (consigliato per sviluppo)
# Sostituisci con il tuo repository URL
GITHUB_REPO = "https://github.com/Giovanni000/Project-Work.git"  # ⚠️ MODIFICA QUESTO!
REPO_DIR = "/content/project"

# Clona repository (se non esiste già)
if not Path(REPO_DIR).exists():
    !git clone {GITHUB_REPO} {REPO_DIR}

# Cambia directory al repository
os.chdir(REPO_DIR)
print(f"Repository directory: {os.getcwd()}")

# Opzione 2: Monta Google Drive solo per i dati (immagini)
from google.colab import drive
drive.mount('/content/drive')

# Path ai dati su Drive
DATA_ROOT = Path("/content/drive/MyDrive/Project Work/Data")
print(f"Data directory: {DATA_ROOT}")

# Path locale (se hai copiato le immagini in locale durante step1/step2)
LOCAL_DATA_DIR = Path("/content/local_data")
print(f"Local data directory: {LOCAL_DATA_DIR}")

# Determina quale path usare (locale se esiste, altrimenti Drive)
if LOCAL_DATA_DIR.exists() and (LOCAL_DATA_DIR / "connectors").exists():
    IMAGE_BASE_PATH = LOCAL_DATA_DIR
    print("✅ Usando immagini in locale (più veloce)")
else:
    IMAGE_BASE_PATH = DATA_ROOT
    print("ℹ️  Usando immagini su Drive")

# Import necessari
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from pathlib import Path
import numpy as np

# Importa classi e funzioni dai notebook precedenti
# Nota: Assicurati di aver eseguito step1 e step2 prima di questo notebook

# Verifica device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")


## Importa Modelli e Classi

**Nota:** Esegui prima i notebook `step1_occlusion_classifier.ipynb` e `step2_autoencoder.ipynb` per avere i modelli addestrati.

Oppure importa le classi se hai i file Python:


In [ ]:
# Importa le classi necessarie
# Se stai usando i file Python:
# from step1_occlusion_classifier import OcclusionCNN
# from step2_autoencoder import ConvAE

# Se stai usando solo i notebook, le classi dovrebbero essere già in memoria
# dopo aver eseguito step1 e step2. Altrimenti, definiscile qui (vedi celle successive)

# Funzioni helper per caricare i modelli
def load_occlusion_model(device=None):
    """Carica il modello classificatore OCCLUSION."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Se OcclusionCNN non è definita, devi importarla o definirla
    # Per ora assumiamo che sia già stata eseguita step1
    try:
        model = OcclusionCNN().to(device)
    except NameError:
        raise NameError("OcclusionCNN non definita. Esegui prima step1_occlusion_classifier.ipynb")
    
    model_path = Path("models/occlusion_cnn.pth")
    if not model_path.exists():
        raise FileNotFoundError(f"Modello non trovato: {model_path}")
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model


def load_ae_and_threshold(device=None):
    """Carica l'autoencoder e il threshold."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Se ConvAE non è definita, devi importarla o definirla
    try:
        model = ConvAE().to(device)
    except NameError:
        raise NameError("ConvAE non definita. Esegui prima step2_autoencoder.ipynb")
    
    model_path = Path("models/ae_conv.pth")
    if not model_path.exists():
        raise FileNotFoundError(f"Modello non trovato: {model_path}")
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    threshold_path = Path("models/ae_threshold.npy")
    if not threshold_path.exists():
        raise FileNotFoundError(f"Threshold non trovato: {threshold_path}")
    
    threshold = np.load(threshold_path)
    return model, threshold


## Funzioni di Preprocessing


In [ ]:
def preprocess_image(image_path, device=None):
    """
    Preprocessa un'immagine per l'inferenza (classificatore OCCLUSION).
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Trasformazioni (stesse del training)
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Carica e preprocessa
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)  # Aggiungi dimensione batch
    image_tensor = image_tensor.to(device)
    
    return image_tensor


def preprocess_image_for_ae(image_path, device=None):
    """
    Preprocessa un'immagine per l'autoencoder (senza normalizzazione ImageNet).
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Trasformazioni (stesse del training AE)
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),  # Già in [0, 1]
    ])
    
    # Carica e preprocessa
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)  # Aggiungi dimensione batch
    image_tensor = image_tensor.to(device)
    
    return image_tensor


## Funzione di Classificazione Principale


In [ ]:
def classify_connector(image_path, device=None):
    """
    Classifica un connettore come "OK", "KO" o "OCCLUSION".
    
    Pipeline:
    1. STEP 1 - Occlusione: Verifica se l'immagine è occlusa
       - Se occlusa → ritorna "OCCLUSION"
    2. STEP 2 - Anomaly Detection: Se visibile, verifica se è anomalo
       - Se errore ricostruzione > threshold → ritorna "KO"
       - Altrimenti → ritorna "OK"
    
    Args:
        image_path: Path all'immagine del connettore
        device: Device (cuda/cpu)
    
    Returns:
        str: "OK", "KO" o "OCCLUSION"
        
    Note:
        - "OCCLUSION" = immagine non leggibile (cavi o altro coprono la zona critica)
        - "OK" = connettore visibile e simile ai campioni OK di training
        - "KO" = connettore visibile ma anomalo rispetto ai campioni OK
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Carica modelli (caricati una volta e riutilizzati)
    try:
        occ_model = load_occlusion_model(device)
        ae_model, threshold = load_ae_and_threshold(device)
    except (FileNotFoundError, NameError) as e:
        raise FileNotFoundError(
            f"Modelli non trovati o classi non definite. "
            f"Assicurati di aver eseguito step1 e step2. Errore: {e}"
        )
    
    # STEP 1: Verifica occlusione
    x_occ = preprocess_image(image_path, device)
    
    with torch.no_grad():
        logits = occ_model(x_occ)
        pred_vis = torch.argmax(logits, dim=1).item()
    
    # Se pred_vis == 0 → OCCLUSION
    if pred_vis == 0:
        return "OCCLUSION"
    
    # STEP 2: Anomaly detection (solo se visibile)
    x_ae = preprocess_image_for_ae(image_path, device)
    
    with torch.no_grad():
        reconstructed = ae_model(x_ae)
        # Calcola errore MSE medio su [C, H, W]
        mse = nn.MSELoss(reduction='mean')
        error = mse(reconstructed, x_ae).item()
    
    # Se errore > threshold → KO (anomalo)
    if error > threshold:
        return "KO"
    else:
        return "OK"


In [ ]:
# I modelli vengono caricati automaticamente da classify_connector()
# Ma se vuoi caricarli manualmente per testare:

# Nota: Le classi OcclusionCNN e ConvAE devono essere in memoria
# (eseguite da step1 e step2) oppure importate dai file Python

# Esempio di caricamento manuale:
# occ_model = load_occlusion_model(device)
# ae_model, threshold = load_ae_and_threshold(device)
# print("Modelli caricati con successo!")


## Test Inferenza

Testa la funzione di classificazione su alcune immagini.


In [ ]:
# Test su alcune immagini di esempio
# Usa IMAGE_BASE_PATH (locale o Drive) per costruire i path corretti
test_filenames = [
    ("conn1", "20251106110559_TOP.png"),
    ("conn2", "20251106110559_TOP.png"),
    ("conn3", "20251106110559_TOP.png"),
]

# Costruisci i path completi
test_images = [
    str(IMAGE_BASE_PATH / "connectors" / connector / filename)
    for connector, filename in test_filenames
]

print("Test classificazione:\n")
print(f"Base path: {IMAGE_BASE_PATH}\n")

for img_path in test_images:
    img_path_obj = Path(img_path)
    if img_path_obj.exists():
        try:
            result = classify_connector(str(img_path_obj), device=device)
            print(f"  {img_path_obj.name} ({img_path_obj.parent.name}): {result}")
        except Exception as e:
            print(f"  {img_path_obj.name}: ❌ ERRORE - {e}")
    else:
        print(f"  {img_path}: ⚠️  Immagine non trovata")
